# Association

In [53]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [175]:
order_df=pd.read_csv('orders.csv')
products_df=pd.read_csv('products.csv')
order_product_prior_df=pd.read_csv('order_products__prior.csv')
order_product_train_df=pd.read_csv('order_products__train.csv')
department_df=pd.read_csv('departments.csv')
aisle_df=pd.read_csv('aisles.csv')

## Test Train Split

In [29]:
order_df[(order_df.eval_set=="test") & (order_df.user_id==3)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0


In [30]:
order_df[(order_df.eval_set=="train") & (order_df.user_id==2)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
25,1492625,2,train,15,1,11,30.0


In [34]:
order_df[(order_df.eval_set=="test")].order_id.count()

75000

In [37]:
order_df[(order_df.eval_set=="train")].order_id.count()

131209

# Test on one user

In [176]:
user_3 = order_df[order_df.user_id==3]

In [177]:
user_3_prior = user_3[user_3.eval_set=='prior']
user_3_test = user_3[user_3.eval_set=='test']

In [178]:
user_3_prior

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
26,1374495,3,prior,1,1,14,NaN
27,444309,3,prior,2,3,19,9.0
28,3002854,3,prior,3,3,16,21.0
29,2037211,3,prior,4,2,18,20.0
30,2710558,3,prior,5,0,17,12.0
31,1972919,3,prior,6,0,16,7.0
32,1839752,3,prior,7,0,15,7.0
33,3225766,3,prior,8,0,17,7.0
34,3160850,3,prior,9,0,16,7.0
35,676467,3,prior,10,3,16,17.0


In [179]:
user_3_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0


In [205]:
df = pd.merge(user_3_prior,order_product_prior_df,on='order_id')
df.sort_values(by=['user_id','order_number'],inplace=True)

In [206]:
df=pd.merge(df,products_df,on='product_id')

In [207]:
df.drop(columns=['order_id','user_id','eval_set','add_to_cart_order','order_dow','order_hour_of_day','reordered','days_since_prior_order','product_id','aisle_id','department_id'],inplace=True)

In [208]:
df

,order_number,product_name
0,1,Granny Smith Apples
1,4,Granny Smith Apples
2,5,Granny Smith Apples
3,6,Granny Smith Apples
4,7,Granny Smith Apples
...,...,...
83,12,Garlic Couscous
84,8,Crunchy Almond Butter
85,8,Birthday Cake Light Ice Cream
86,9,Birthday Cake Light Ice Cream


In [209]:
df = pd.get_dummies(df, columns=['product_name'],prefix='', prefix_sep='')

In [210]:
df=df.groupby('order_number').sum()

In [211]:
df

,100% Recycled Paper Towels,All Natural No Stir Creamy Almond Butter,Birthday Cake Light Ice Cream,Blueberry Pint,Brussels Sprouts,Celery Sticks,China Green Tips Green Tea,Chocolate Ice Cream,Chopped Walnuts,"Crackers, Crispy, Cheddar",...,Organic Shredded Mozzarella,Organic Strawberries,Organic Whole String Cheese,Original Rotisserie Chicken,Part Skim Ricotta Cheese,Strawberries,Unsweetened Chocolate Almond Breeze Almond Milk,Vanilla Unsweetened Almond Milk,Wheat Gluten Free Waffles,Whole Almonds
order_number,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,1,1,1,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
4,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,1,0,0,1,0,0,...,1,0,1,1,0,0,1,1,0,0
6,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
9,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


## Apriori

In [212]:
frequent_item = apriori(df,min_support=0.01,use_colnames=True)
frequent_item["length"] = frequent_item["itemsets"].apply(lambda x: len(x))

In [221]:
frequent_item[frequent_item.length>=10]

,support,itemsets,length
4628,0.083333,"(Celery Sticks, Organic Baby Spinach, Strawber...",10
4629,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10
4630,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10
4631,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10
4632,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10
4633,0.083333,"(Organic Baby Spinach, Granny Smith Apples, Or...",10
4634,0.083333,"(Organic Shredded Mozzarella, Granny Smith App...",10
4635,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10
4636,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10
4637,0.083333,"(Organic Baby Spinach, Organic Shredded Mozzar...",10


### Confidence

In [222]:
rules = association_rules(frequent_item,metric="confidence",min_threshold=0.5)

In [223]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100% Recycled Paper Towels),"(Crackers, Crispy, Cheddar)",0.083333,0.250000,0.083333,1.0,4.0,0.062500,inf
1,(Dried Sweetened Cranberries),(100% Recycled Paper Towels),0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
2,(100% Recycled Paper Towels),(Dried Sweetened Cranberries),0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
3,(Electrolyte Water),(100% Recycled Paper Towels),0.166667,0.083333,0.083333,0.5,6.0,0.069444,1.833333
4,(100% Recycled Paper Towels),(Electrolyte Water),0.083333,0.166667,0.083333,1.0,6.0,0.069444,inf


In [224]:
rules = association_rules(frequent_item,metric="lift",min_threshold=0.5)

In [225]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Crackers, Crispy, Cheddar)",(100% Recycled Paper Towels),0.250000,0.083333,0.083333,0.333333,4.0,0.062500,1.375000
1,(100% Recycled Paper Towels),"(Crackers, Crispy, Cheddar)",0.083333,0.250000,0.083333,1.000000,4.0,0.062500,inf
2,(Dried Sweetened Cranberries),(100% Recycled Paper Towels),0.083333,0.083333,0.083333,1.000000,12.0,0.076389,inf
3,(100% Recycled Paper Towels),(Dried Sweetened Cranberries),0.083333,0.083333,0.083333,1.000000,12.0,0.076389,inf
4,(Electrolyte Water),(100% Recycled Paper Towels),0.166667,0.083333,0.083333,0.500000,6.0,0.069444,1.833333


In [233]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))

In [246]:
rules[(rules['confidence']>0.8) & (rules['lift']>6) & (rules['antecedent_len']>=5) & (rules['consequents_len']>=5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequents_len
266911,"(Celery Sticks, Organic Baby Spinach, Strawber...","(Blueberry Pint, Organic Avocado, Vanilla Unsw...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,5
266912,"(Celery Sticks, Organic Baby Spinach, Strawber...","(Organic Lightly Salted Brown Rice Cakes, Orga...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,5
266913,"(Celery Sticks, Organic Baby Spinach, Strawber...","(Organic Lightly Salted Brown Rice Cakes, Blue...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,5
266914,"(Celery Sticks, Organic Baby Spinach, Strawber...","(Organic Lightly Salted Brown Rice Cakes, Blue...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,5
266915,"(Celery Sticks, Organic Baby Spinach, Strawber...","(Organic Lightly Salted Brown Rice Cakes, Blue...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,5
...,...,...,...,...,...,...,...,...,...,...,...
280270,"(Vanilla Unsweetened Almond Milk, Original Rot...","(Organic Baby Spinach, Organic Shredded Mozzar...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,6
280271,"(Chocolate Ice Cream, Vanilla Unsweetened Almo...","(Organic Baby Spinach, Organic Shredded Mozzar...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,6
280272,"(Chocolate Ice Cream, Vanilla Unsweetened Almo...","(Organic Baby Spinach, Organic Shredded Mozzar...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,6
280273,"(Chocolate Ice Cream, Original Rotisserie Chic...","(Organic Baby Spinach, Organic Shredded Mozzar...",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf,5,6
